In [3]:
import time
from euler import EulerPeriodicDataset

h5_path = "/scratch/izar/boscario/datasets/euler_multi_quadrants_periodicBC/data/train/euler_multi_quadrants_periodicBC_gamma_1.76_Ar_-180.hdf5"
stats_path = "/scratch/izar/boscario/datasets/euler_multi_quadrants_periodicBC/stats.yaml"  

# instantiate
ds = EulerPeriodicDataset(h5_path, stats_path=stats_path, time_window=1, patch_size=None, normalize=True)
print("Dataset created:", len(ds))

# basic metadata checks
print("len(dataset) =", len(ds))
print("n_sims, n_t, H, W =", ds.n_sims, ds.n_t, ds.H, ds.W)
print("n_per_sim =", ds.n_per_sim, "total_samples =", ds.total_samples)

# check cached static entries
print("static cache keys:", list(ds._static_cache.keys()))
print("x_periodic, y_periodic:", ds._static_cache.get("x_periodic"), ds._static_cache.get("y_periodic"))
print("gamma:", ds._static_cache.get("gamma"))
print("pos_template shape:", ds._static_cache.get("pos_template").shape)

# --- Fast I/O test: load arrays only (no edge build) ---
arrs = ds._load_time_window(sim_idx=0, t_idx=0)
print("density shape:", arrs["density"].shape)   # expect (time_window, H, W)
print("momentum shape:", arrs["momentum"].shape) # expect (time_window, H, W, 2)

# --- Build the graph for a single sample (this triggers edge construction; may take a few seconds and memory) ---
t0 = time.time()
data = ds[0]   # <-- heavy: creates full-grid edges & attributes
t1 = time.time()
print("Built Data object in {:.1f}s".format(t1-t0))
print("data.x shape (N, C):", data.x.shape)
print("data.pos shape:", data.pos.shape)
print("edge_index shape:", data.edge_index.shape)
print("edge_attr shape:", data.edge_attr.shape)
print("global features u:", data.u)


EulerPeriodicDataset: using full-grid samples (512x512) — this is large (262144 nodes). Consider patching.
Dataset created: 40000
len(dataset) = 40000
n_sims, n_t, H, W = 400 101 512 512
n_per_sim = 100 total_samples = 40000
static cache keys: ['pos_template', 'x_coords', 'y_coords', 'gamma', 'x_periodic_mask', 'y_periodic_mask', 'x_periodic', 'y_periodic']
x_periodic, y_periodic: True True
gamma: 1.7599999904632568
pos_template shape: (262144, 2)
density shape: (1, 512, 512)
momentum shape: (1, 512, 512, 2)
Built Data object in 5.0s
data.x shape (N, C): torch.Size([262144, 5])
data.pos shape: torch.Size([262144, 2])
edge_index shape: torch.Size([2, 1048576])
edge_attr shape: torch.Size([1048576, 4])
global features u: tensor([[1.7600, 0.0000]])


In [ ]:
import numpy as np

def compute_mean_fields(ds):
    """
    Computes the mean fields values over the entire dataset (all simulations, all timesteps).
    """
    mean_density = []
    mean_pressure = []
    mean_energy = []
    mean_momentum = []  # magnitude

    for i in range(ds.n_sims):
        # load all time possible steps for simulation i
        arrs = ds._load_time_window(sim_idx=i, t_idx=0)
        mean_density.append(arrs["density"].mean())
        mean_pressure.append(arrs["pressure"].mean())
        mean_energy.append(arrs["energy"].mean())

        # for momentum, compute magnitude first
        arrs["momentum"] = np.sqrt((arrs["momentum"]**2).sum(axis=-1))
        mean_momentum.append(arrs["momentum"].mean())

    # now compute global means
    mean_density = sum(mean_density) / ds.n_sims
    mean_pressure = sum(mean_pressure) / ds.n_sims
    mean_energy = sum(mean_energy) / ds.n_sims
    mean_momentum = sum(mean_momentum) / ds.n_sims

    return mean_density, mean_pressure, mean_energy, mean_momentum

In [ ]:
train_path = "/scratch/izar/boscario/datasets/euler_multi_quadrants_periodicBC/data/train/euler_multi_quadrants_periodicBC_gamma_1.4_Dry_air_20.hdf5"
valid_path = "/scratch/izar/boscario/datasets/euler_multi_quadrants_periodicBC/data/valid/euler_multi_quadrants_periodicBC_gamma_1.4_Dry_air_20.hdf5"
test_path = "/scratch/izar/boscario/datasets/euler_multi_quadrants_periodicBC/data/test/euler_multi_quadrants_periodicBC_gamma_1.4_Dry_air_20.hdf5"

train_ds = EulerPeriodicDataset(train_path, stats_path=stats_path, time_window=1, patch_size=None, normalize=True)
valid_ds = EulerPeriodicDataset(valid_path, stats_path=stats_path, time_window=1, patch_size=None, normalize=True)
test_ds = EulerPeriodicDataset(test_path, stats_path=stats_path, time_window=1, patch_size=None, normalize=True)

# compute mean fields for each split
train_means = compute_mean_fields(train_ds)
valid_means = compute_mean_fields(valid_ds)
test_means = compute_mean_fields(test_ds)

print("Train means (density, pressure, energy, momentum):", train_means)
print("Valid means (density, pressure, energy, momentum):", valid_means)
print("Test means (density, pressure, energy, momentum):", test_means)

EulerPeriodicDataset: using full-grid samples (512x512) — this is large (262144 nodes). Consider patching.
EulerPeriodicDataset: using full-grid samples (512x512) — this is large (262144 nodes). Consider patching.
EulerPeriodicDataset: using full-grid samples (512x512) — this is large (262144 nodes). Consider patching.
Train means (density, pressure, momentum, energy): (0.9299198387563229, 0.9002689383178949, 0.5267369353398681, 2.459686482846737)
Valid means (density, pressure, momentum, energy): (0.9133071875572205, 0.8708313792943955, 0.4901703608036041, 2.3710775804519653)
Test means (density, pressure, momentum, energy): (0.8470769155025483, 0.7551606976985932, 0.48677064090967176, 2.0858439445495605)
